In [1]:
from Datos import Datos
from Arm import ArmkNN
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
knn = ArmkNN(20)


In [4]:
# Prueba de kNN
train, test = train_test_split(kNNdataset, train_size=0.1)
knn.initSet(train)

num = 0
den = 0

for u in listUsers:
    item = knn.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        knn.add_sample(hit[0])
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        knn.add_bad_sample(u,item)

1.0
1.0
1.0
1.0
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.8823529411764706
0.8888888888888888
0.8947368421052632
0.9
0.9047619047619048
0.9090909090909091
0.9130434782608695
0.9166666666666666
0.92
0.9230769230769231
0.9259259259259259
0.9285714285714286
0.9310344827586207
0.9333333333333333
0.9032258064516129
0.90625
0.9090909090909091
0.8823529411764706
0.8857142857142857
0.8888888888888888
0.8918918918918919
0.8947368421052632
0.8974358974358975
0.9
0.9024390243902439
0.9047619047619048
0.9069767441860465
0.9090909090909091
0.9111111111111111
0.9130434782608695
0.9148936170212766
0.9166666666666666
0.9183673469387755
0.92
0.9215686274509803
0.9230769230769231
0.9245283018867925
0.9259259259259259
0.9272727272727272
0.9285714285714286
0.9122807017543859
0.9137931034482759
0.9152542372881356
0.9
0.9016393442622951
0.9032258064516129
0.9047619047619048
0.

Pruebas de NB

In [3]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split


dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

train, test = train_test_split(kNNdataset, train_size=0.1)

arm = ArmNB()
arm.initSet(train)

arm.rec_item(1)

6753.0

In [10]:
from Datos import Datos
from Arm import ArmNB
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)


listUsers = np.unique(kNNdataset[:,0])
train, test = train_test_split(kNNdataset, train_size=0.1)
arm = ArmNB(train)

In [11]:
# Prueba de NB


num = 0
den = 0

for u in listUsers:
    item = arm.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        train = np.vstack((train,hit))
        den += 1
        if(hit[0,2] >= 3):
            num += 1
        print(num/den)
    else:
        train = np.vstack((train,[u,item,1.]))

1.0
1.0
1.0
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.9444444444444444
0.9473684210526315
0.9
0.9047619047619048
0.9090909090909091
0.9130434782608695
0.9166666666666666
0.92
0.9230769230769231
0.9259259259259259
0.9285714285714286
0.9310344827586207
0.9
0.9032258064516129
0.90625
0.9090909090909091
0.9117647058823529
0.9142857142857143
0.9166666666666666
0.918918918918919
0.9210526315789473
0.9230769230769231
0.925
0.926829268292683
0.9285714285714286
0.9302325581395349
0.9318181818181818
0.9333333333333333
0.9347826086956522
0.9361702127659575
0.9375
0.9387755102040817
0.94
0.9411764705882353
0.9423076923076923
0.9433962264150944
0.9444444444444444
0.9454545454545454
0.9464285714285714
0.9473684210526315
0.9482758620689655
0.9491525423728814
0.95
0.9508196721311475
0.9516129032258065
0.9523809523809523
0.953125
0.9538461538461539

Probando cosas más chungas

In [2]:
from Datos import Datos
from Arm import ArmItemNB
import numpy as np

from sklearn.model_selection import train_test_split

dataset = Datos("ratings.csv")

kNNcols = ['userId','movieId','rating']
kNNdataset = dataset.extraeCols(kNNcols)

datasetM = Datos("movies.csv",nominal=False)

cols = ['movieId','genres']
npdatasetM = datasetM.extraeCols(cols)

train, test = train_test_split(kNNdataset, train_size=0.1)
listUsers = np.unique(train[:,0])

arm = ArmItemNB(npdatasetM)
arm.initSet(train)

In [4]:
# Prueba de NB con items


num = 0
den = 0

for u in listUsers:
    item = arm.rec_item(u)
    # Ejemplo encontrado para la recomendación
    mask1 = test[:,0] == u
    mask2 = test[:,1] == item
    mask = np.logical_and(mask1,mask2)
    test, hit = test[np.logical_not(mask)], test[mask]
    if(len(hit) != 0):
        den += 1
        if(hit[0,2] >= 3):
            rt = 1
            num += 1
        else:
            rt = -1
        print(num/den)
        arm.add_sample(hit[0])
    else:
        arm.add_sample([u,item,-1])

1.0
1.0
1.0
0.75
0.8
0.8333333333333334
0.8571428571428571
0.875
0.8888888888888888
0.9
0.9090909090909091
0.9166666666666666
0.9230769230769231
0.9285714285714286
0.9333333333333333
0.9375
0.9411764705882353
0.8888888888888888
0.8947368421052632
0.9
0.9047619047619048
0.9090909090909091
0.9130434782608695
0.9166666666666666
0.92
0.9230769230769231
0.9259259259259259
0.9285714285714286
0.9310344827586207
0.9333333333333333
0.9354838709677419
0.9375
0.9393939393939394
0.9411764705882353
0.9428571428571428
0.9444444444444444
0.9459459459459459
0.9473684210526315
0.9487179487179487
0.95
0.9512195121951219
0.9523809523809523
0.9302325581395349
0.9318181818181818
0.9333333333333333


Probando el modelo final

In [4]:
from Bandit import epsilonGreedy

bandit = epsilonGreedy("ratings.csv","movies.csv")

bandit.add_knnArm(2)
bandit.add_NB()

bandit.run_epoch(epochs = 100)

for arm in bandit.arms:
    print(arm.accuracy())
    print(arm.fails + arm.hits)

1.0
16
0
0
